<font color='green' size='6'>**Практическая работа 2**</font>  
<font color='#11a642' size='5'>**Тема: Машинное обучение**</font>  
<font size='5'>**Задание:**</font>  

Домашняя работа основана на данных конкурса https://www.kaggle.com/competitions/playground-series-s4e10/

**Ваша задача** — предсказать, останется ли клиент с аккаунтом или закроет его (например, уйдёт).

Требования к оформлению:
- Работа должна быть оформлена в jupyter notebook (файл *.ipynb)
- Работа должна содержать комментарии и выводы. Объясняйте почему вы выбрали именно этот способ обработки данных


Всем удачи!





# <font color='#11a642' size='6'> **Импорт и установка библиотек**

In [1]:
import pandas as pd

# <font color='#11a642' size='6'> **Загрузка данных**

In [2]:
# 1. Загрузка датасета
!gdown 1wvqGslWLauKyZxLY-d3RQCTyHCb5wJCO

Downloading...
From: https://drive.google.com/uc?id=1wvqGslWLauKyZxLY-d3RQCTyHCb5wJCO
To: c:\Users\alexander\Desktop\python_innopolis_ml\part_2\practice_work\train.csv

  0%|          | 0.00/12.4M [00:00<?, ?B/s]
  4%|▍         | 524k/12.4M [00:00<00:03, 3.24MB/s]
 25%|██▌       | 3.15M/12.4M [00:00<00:00, 13.9MB/s]
100%|██████████| 12.4M/12.4M [00:00<00:00, 34.2MB/s]


In [ ]:
df = pd.read_csv('part_2\content\train.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/train.csv'

In [ ]:
df.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,1,15749177,Okwudiliolisa,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,2,15694510,Hsueh,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,3,15741417,Kao,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,4,15766172,Chiemenam,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0


In [ ]:
# 2. Разделение признаков и целевой переменной
X = df.drop('Exited', axis=1)
y = df['Exited']

In [ ]:
y.head()

,Exited
0,0
1,0
2,0
3,0
4,0


## <font color='#11a642' size='5'> Напишите ответы на следующие вопросы: </font>


1.   К какому типу переменных относится целевой признак?
2.   Какую задачу машинного обучения вам предстоит решить, регрессия или классификация?

# ваш ответ

# <font color='#11a642' size='6'> **Разведочный анализ данных**

## <font color='#11a642' size='5'> Оцените размер вашего датасета  </font>



1.   Сколько ГБ ваш датасет
2.   Сколько строк и столбцов



In [ ]:
# Сколько ГБ ваш датасет
# ваш код

In [ ]:
# Сколько строк и столбцов
# ваш код

## <font color='#11a642' size='5'> Анализ числовых (непрерывных) признаков


- для каждого признака посчитайте минимум, максимум, среднее, медиану

In [ ]:
# ваш код

## <font color='#11a642' size='5'> Анализ категориальных признаков


- для каждого признака посчитайте моду, количество уникальных значений

In [ ]:
# ваш код

## <font color='#11a642' size='5'> Анализ целевого признака


- какая доля 1?

In [ ]:
# ваш код

# <font color='#11a642' size='6'> **Разделите данные на трейн и тест**

- нужна ли стратификация?

In [ ]:
# ваш код

# <font color='#11a642' size='6'> **Подготовка данных к обучению**

## Поиск и анализ выбросов

In [ ]:
# ваш код

## Анализ и обработка пропусков

In [ ]:
# ваш код

## Анализ и обработка категориальных переменных

In [ ]:
# ваш код

## Масштабирование числовых признаков

In [ ]:
# ваш код

# <font color='#11a642' size='6'> **Построение моделей машинного обучения**

## <font color='#11a642' size='5'> Построение логистической регрессии

In [ ]:
# ваш код

## <font color='#11a642' size='5'> Построение логистической регрессии с регуляризацией
> можно использовать [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression), настроив гиперпараметры penalty и C

In [ ]:
# ваш код

### <font color='#11a642' size='4'> Оценка качества  всех моделей


In [ ]:
# ваш код

# <font color='#11a642' size='4'> Ваши выводы
- какую из моделей вы рекомендуете использовать для прогноза? Почему?



ВАШ ОТВЕТ